# AlexNet

AlexNet与LeNet网络架构基本相似，只是AlexNet更深，使用了8层卷积神经网络（5个卷积层，3个最大汇聚层），并且使用的激活函数不再是Sigmoid而是ReLU，其具体[网络架构](../image/AlexNet.jpg)如下:
<div align="center">
<img src="https://s2.loli.net/2022/04/12/woAXRiu9qeG4l3P.jpg" width="30%">
<br>LeNet架构图（左）AlexNet架构图（右）
</div>

基于pytorch实现AlexNet网络的模型搭建如下：

## 模型建立

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class AlexNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(AlexNet, self).__init__()
        """
        num_classes:分类的数量
        in_channels:原始图像通道数,灰度图像为1,彩色图像为3
        """
        self.num_classes = num_classes
        self.lenet = nn.Sequential(
            nn.Conv2d(in_channels, 96, kernel_size=11, stride=4, padding=2),  # (224 - 11 + 4) / 4 + 1 = 55.25   [3, 224, 224] -> [96, 55, 55]
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),  # (55 + 2 - 3) / 2 = 27   [96, 55, 55] -> [96, 27, 27]
            nn.Conv2d(96, 256, kernel_size=5, padding=2),  # (27 - 5 + 4) / 1 + 1 = 27   [96, 27, 27] -> [256, 27, 27]
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2), # (27 - 3 + 2) / 2 = 13   [256, 27, 27] -> [256, 13, 13]
            nn.Conv2d(256, 384, kernel_size=3, padding=1),  # (13 - 3 + 2) / 1 + 1 = 13   [256, 13, 13] -> [384, 13, 13]
            nn.ReLU(),
            nn.Conv2d(384, 384, kernel_size=3, padding=1),  # (13 - 3 + 2) / 1 + 1 = 13   [384, 13, 13] -> [384, 13, 13]
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),  # (13 - 3 + 2) / 1 + 1 = 13   [384, 13, 13] -> [256, 13, 13]
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2), # (13 - 3 + 2) / 2 = 6   [256, 13, 13] -> [256, 6, 6]
            nn.Flatten(), # 展平
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes)) # 不需要使用激活函数，因为softmax激活函数被嵌入在交叉熵函数中

    def forward(self, x):
        output = self.lenet(x)
        # probas = F.softmax(output, dim=1)
        return output #, probas

在第一次卷积操作时，不能完全便利整个图像，应该在左、上添两列零，右、下添一列零能完全遍历；使用padding=2填充，在操作过程中会自动省去多余数据，故影响不大。

## 模型检验

打印出各层的参数

In [2]:
num_classes = 1000
in_channels = 3
X = torch.rand(size=(1, in_channels, 224, 224), dtype=torch.float32)
net = AlexNet(in_channels, num_classes)
for layer in net.lenet:
  X = layer(X)
  print(layer.__class__.__name__,'output shape: \t',X.shape)

Conv2d output shape: 	 torch.Size([1, 96, 55, 55])
ReLU output shape: 	 torch.Size([1, 96, 55, 55])
MaxPool2d output shape: 	 torch.Size([1, 96, 27, 27])
Conv2d output shape: 	 torch.Size([1, 256, 27, 27])
ReLU output shape: 	 torch.Size([1, 256, 27, 27])
MaxPool2d output shape: 	 torch.Size([1, 256, 13, 13])
Conv2d output shape: 	 torch.Size([1, 384, 13, 13])
ReLU output shape: 	 torch.Size([1, 384, 13, 13])
Conv2d output shape: 	 torch.Size([1, 384, 13, 13])
ReLU output shape: 	 torch.Size([1, 384, 13, 13])
Conv2d output shape: 	 torch.Size([1, 256, 13, 13])
ReLU output shape: 	 torch.Size([1, 256, 13, 13])
MaxPool2d output shape: 	 torch.Size([1, 256, 6, 6])
Flatten output shape: 	 torch.Size([1, 9216])
Linear output shape: 	 torch.Size([1, 4096])
ReLU output shape: 	 torch.Size([1, 4096])
Dropout output shape: 	 torch.Size([1, 4096])
Linear output shape: 	 torch.Size([1, 4096])
ReLU output shape: 	 torch.Size([1, 4096])
Dropout output shape: 	 torch.Size([1, 4096])
Linear output sh

## 模型训练

*后补*

## 模型检测

*后补*

### AlexNet与LeNet不同

* 网络更深，使用8层网络进行训练
* 激活函数改进，使用ReLU替代Sigmoid
* 池化层采用Maxpool2d代替Avgpool2d
* 在全连接层处使用Dropout丢弃法